# Analisi del Sentiment sulle Traduzioni del Dataset ASLLRP

**Obiettivo dello script:**
Questo script ha l'obiettivo di eseguire un'analisi del sentiment sulle traduzioni in inglese delle frasi in Lingua dei Segni Americana (ASL) provenienti dal dataset ASLLRP. Lo scopo è classificare ogni frase come "Positiva", "Negativa" o "Neutra" per creare un dataset etichettato che possa essere utilizzato per addestrare modelli di riconoscimento delle emozioni dai video.

**Funzionamento dello script:**

1.  **Caricamento e Pulizia dei Dati:**

    - Lo script inizia caricando il file `utterances_with_translations.csv`, che contiene i nomi dei file video e le relative traduzioni in inglese (generate dallo script precedente `03_matching_videos_utterances_ASLLRP.ipynb`).
    - Successivamente, esegue una fase di pulizia per rimuovere le righe in cui la traduzione non è valida o mancante (es. "Traduzione non trovata" o "Translation tag vuota"). Questo assicura che l'analisi del sentiment venga eseguita solo su dati di qualità.

2.  **Analisi del Sentiment con VADER:**

    - Per ogni traduzione (caption), viene utilizzata la libreria **VADER (Valence Aware Dictionary and sEntiment Reasoner)**. VADER è uno strumento specifico per l'analisi del sentiment su testi provenienti da social media e contesti informali, ma si dimostra efficace anche per frasi generiche.
    - VADER calcola un `compound_score`, un punteggio normalizzato compreso tra -1 (massimamente negativo) e +1 (massimamente positivo).

3.  **Classificazione delle Emozioni:**

    - Il `compound_score` viene utilizzato per classificare ogni frase in una delle tre categorie emotive:
      - **Positiva:** se il `compound_score` è superiore a una soglia predefinita (es. `0.34`).
      - **Negativa:** se il `compound_score` è inferiore a una soglia negativa (es. `-0.34`).
      - **Neutra:** se il punteggio si trova tra le due soglie.
    - La soglia (`threshold`) è un parametro chiave che può essere modificato per calibrare la sensibilità della classificazione.

4.  **Visualizzazione e Salvataggio:**
    - Viene generato un grafico a barre che mostra la distribuzione delle frasi nelle tre categorie (Positiva, Negativa, Neutra), permettendo una rapida valutazione del bilanciamento del dataset.
    - I risultati finali, che includono il nome del video, la traduzione e l'etichetta di emozione, vengono salvati in un nuovo file CSV. Questo file (`asllrp_video_sentiment_data_with_neutral_{threshold}.csv`) servirà come base per le fasi successive del progetto.


In [15]:
# Step 1: Importare le librerie necessarie
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import os
from collections import Counter
import matplotlib

# Imposta il backend di Matplotlib per evitare problemi di visualizzazione in alcuni ambienti
matplotlib.use("Agg")

In [16]:
# Step 2: Caricare e pulire i dati
# Percorso del file CSV
csv_path = "../data/processed/utterances_with_translations.csv"

# Carica il file CSV
df = pd.read_csv(csv_path)

# Mostra le prime righe e le informazioni sul DataFrame
# print("Prime 5 righe del dataset:")
# print(df.head())
print(f"\nColonne disponibili: {list(df.columns)}")
print(f"Numero totale di righe: {len(df)}")

# Rimuovi le righe dove la traduzione non è stata trovata, è vuota o non è una stringa valida
df_cleaned = df.dropna(subset=["caption"])
df_cleaned = df_cleaned[
    ~df_cleaned["caption"].str.contains(
        "Traduzione non trovata|Translation tag vuota", na=False
    )
]

print(f"\nNumero di righe dopo la pulizia: {len(df_cleaned)}")
# print("Prime 5 righe del dataset pulito:")
# print(df_cleaned.head())


Colonne disponibili: ['video_name', 'Source collection', 'caption']
Numero totale di righe: 2127

Numero di righe dopo la pulizia: 2118


In [17]:
# Identifica e visualizza le righe scartate
# Le righe scartate sono quelle presenti in 'df' ma non in 'df_cleaned'
# Possiamo trovarle prendendo l'indice di 'df' e rimuovendo gli indici di 'df_cleaned'
discarded_indices = df.index.difference(df_cleaned.index)
discarded_rows = df.loc[discarded_indices]

print("Visualizzazione delle righe scartate:")
if not discarded_rows.empty:
    print(discarded_rows)
else:
    print("Nessuna riga è stata scartata.")

Visualizzazione delle righe scartate:
        video_name         Source collection                 caption
30    45035462.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
31    45107545.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
32    45239274.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
33    45404906.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
1306   8396360.mp4      Cory_2013-6-27_sc109                     NaN
1311   8395965.mp4      Cory_2013-6-27_sc109                     NaN
1627  98357949.mp4          3-Ben-Voice-Life  Traduzione non trovata
1988   3166107.mp4  Jonathan_2012-11-27_sc96                     NaN
2076   7454155.mp4    Rachel_2011-12-08_sc46                     NaN


In [18]:
# Step 3: Analisi del Sentiment
# Inizializza VADER
analyzer = SentimentIntensityAnalyzer()

# Soglia per classificare come positivo o negativo
threshold = 0.5


# Funzione per calcolare il sentiment e classificare l'emozione
def get_sentiment_emotion(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores["compound"]

    if compound_score >= threshold:
        emotion = "Positive"
    elif compound_score <= -threshold:
        emotion = "Negative"
    else:
        emotion = "Neutral"

    return compound_score, emotion


# Applica la funzione al DataFrame
df_cleaned[["compound_score", "emotion"]] = df_cleaned["caption"].apply(
    lambda x: pd.Series(get_sentiment_emotion(x))
)

print("\nPrime 5 righe con punteggi di sentiment ed emozione:")
print(df_cleaned.head())


Prime 5 righe con punteggi di sentiment ed emozione:
     video_name      Source collection  \
0   2751812.mp4  Cory_2013-6-25_sc0106   
1  98592049.mp4  Cory_2013-6-25_sc0106   
2  98857459.mp4  Cory_2013-6-25_sc0106   
3  98881709.mp4  Cory_2013-6-25_sc0106   
4  98895873.mp4  Cory_2013-6-25_sc0106   

                                             caption  compound_score  emotion  
0            There's something strange about myself.         -0.2023  Neutral  
1                        I never lived in a house...          0.0000  Neutral  
2                             I was born in Florida.          0.0000  Neutral  
3  I lived there in a house with my parents and t...          0.0000  Neutral  
4  ... while I was growing up until I was six  an...          0.1779  Neutral  


In [19]:
# Step 4: Visualizzare la distribuzione dei sentiment
# Conta le occorrenze di ogni emozione
emotion_counts = Counter(df_cleaned["emotion"])

print("\nDistribuzione dei sentiment:")
print(f"Positive: {emotion_counts.get('Positive', 0)} esempi")
print(f"Negative: {emotion_counts.get('Negative', 0)} esempi")
print(f"Neutral: {emotion_counts.get('Neutral', 0)} esempi")

# Crea il grafico a barre
plt.figure(figsize=(10, 6))
bars = plt.bar(
    emotion_counts.keys(), emotion_counts.values(), color=["green", "red", "gray"]
)
plt.title("Distribuzione delle Emozioni nel Dataset ASLLRP")
plt.xlabel("Emozione")
plt.ylabel("Numero di Video")

# Aggiungi il numero sopra ogni barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

# Aggiungi la soglia usata come testo
plt.text(
    0.95,
    0.95,
    f"Threshold: |{threshold}|",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    horizontalalignment="right",
    bbox=dict(facecolor="white", alpha=0.5),
)

# Salva l'immagine
output_folder = "../reports/figures"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(
    output_folder, f"asllrp_emotions_distribution_{threshold}.png"
)
plt.savefig(output_path)

print(f"\nGrafico della distribuzione salvato in: {output_path}")
plt.show()


Distribuzione dei sentiment:
Positive: 270 esempi
Negative: 110 esempi
Neutral: 1738 esempi

Grafico della distribuzione salvato in: ../reports/figures/asllrp_emotions_distribution_0.5.png


/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_32184/1347181616.py:52: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [20]:
# Calcola le percentuali di positivi e negativi (escludendo i neutri)
positive_count = emotion_counts.get("Positive", 0)
negative_count = emotion_counts.get("Negative", 0)
total_non_neutral = positive_count + negative_count

if total_non_neutral > 0:
    positive_percentage = (positive_count / total_non_neutral) * 100
    negative_percentage = (negative_count / total_non_neutral) * 100

    print("\nPercentuale di sentiment (esclusi i neutri):")
    print(f"Positivi: {positive_percentage:.2f}%")
    print(f"Negativi: {negative_percentage:.2f}%")
else:
    print("\nNessun esempio positivo o negativo trovato per calcolare la percentuale.")


Percentuale di sentiment (esclusi i neutri):
Positivi: 71.05%
Negativi: 28.95%


In [21]:
# Step 5: Salvare i risultati

# !Codice precedente: Filtra solo i dati positivi e negativi
# filtered_df = df_cleaned[df_cleaned["emotion"].isin(["Positive", "Negative"])]
#
# # Seleziona le colonne da salvare
# output_df = filtered_df[["video_name", "caption", "emotion"]]
#
# # Percorso del file CSV di output
# output_csv_path = f"../data/processed/asllrp_video_sentiment_data_{threshold}.csv"
#
# # Salva il DataFrame in un file CSV
# output_df.to_csv(output_csv_path, index=False)
#
# print(
#     f"\nFile CSV con i dati di sentiment (Positivi e Negativi) salvato con successo in: {output_csv_path}"
# )
# print(f"Numero di righe salvate: {len(output_df)}")


# !Nuovo codice: Salva tutti i dati (inclusi i neutri)
# Seleziona le colonne da salvare
output_df = df_cleaned[["video_name", "caption", "emotion"]]

# Percorso del file CSV di output
output_csv_path = (
    f"../data/processed/asllrp_video_sentiment_data_with_neutral_{threshold}.csv"
)

# Salva il DataFrame in un file CSV
output_df.to_csv(output_csv_path, index=False)

print(
    f"\nFile CSV con i dati di sentiment (Positivi, Negativi e Neutri) salvato con successo in: {output_csv_path}"
)
print(f"Numero di righe salvate: {len(output_df)}")


File CSV con i dati di sentiment (Positivi, Negativi e Neutri) salvato con successo in: ../data/processed/asllrp_video_sentiment_data_with_neutral_0.5.csv
Numero di righe salvate: 2118
